<a href="https://colab.research.google.com/github/fatemekhanipour11/DataBase/blob/main/Python_MySQL_Data_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Preparing the environment to work with MySQL in Google Colab**

In [ ]:
# This line installs the mysql-connector-python package, which allows Python to connect and interact with MySQL databases.
!pip install mysql-connector-python

In [ ]:
# This line installs the MySQL server on the system, enabling the creation and management of MySQL databases locally.
!apt-get -y install mysql-server

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
mysql-server is already the newest version (8.0.39-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
# This line starts the MySQL server service, allowing it to accept connections and perform database operations.
!service mysql start

 * Starting MySQL database server mysqld
   ...done.


In [ ]:
# This line modifies the authentication method and password for the MySQL root user and refreshes the privileges, enabling root access with the specified password.
!mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH 'mysql_native_password' BY 'root';FLUSH PRIVILEGES;"

ERROR 1045 (28000): Access denied for user 'root'@'localhost' (using password: NO)


# **2. Connect to Server and Create Database**



In [ ]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

This function, create_server_connection, establishes a connection to a MySQL database server. It accepts four parameters: host_name (the server's hostname or IP address), user_name (the MySQL username), user_password (the MySQL password), and an optional db_name (the name of the database to connect to). If a database name is provided, the function connects directly to that database. If not, it connects to the server without selecting a specific database. It returns a MySQL connection object if the connection is successful, and prints an error message if it fails.

In [ ]:
def create_server_connection(host_name, user_name, user_password, db_name=None):
    connection = None
    try:
        if db_name:
            connection = mysql.connector.connect(
                host=host_name,
                user=user_name,
                passwd=user_password,
                database=db_name
            )
        else:
            connection = mysql.connector.connect(
                host=host_name,
                user=user_name,
                passwd=user_password
            )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")
    return connection

In [ ]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

def fetch_query_results(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        if result:
            for row in result:
                print(row)
        else:
            print("No results to display.")
        print("\n")
    except Error as err:
        print(f"Error: '{err}'")



In [ ]:
# Connection information with the server
host_name = "localhost"
user_name = "root"
user_password = "root"
db_name = "customers"

# Connect to the server
connection = create_server_connection(host_name, user_name, user_password)


# Create database
create_database_query = f"CREATE DATABASE IF NOT EXISTS {db_name}"
execute_query(connection, create_database_query)

MySQL Database connection successful
Query successful


In [ ]:
# Display the database
databases = fetch_query_results(connection, "SHOW DATABASES")
if databases:
    print("Databases:")
    for db in databases:
        print(db[0])

('customers',)
('information_schema',)
('mysql',)
('performance_schema',)
('sys',)




In [ ]:
# Connect to the new database
connection = create_server_connection(host_name, user_name, user_password, db_name)

MySQL Database connection successful



# **3. Creating Tables**

In [ ]:
# Create customers table
create_customers_table_query = """
CREATE TABLE IF NOT EXISTS customers (
    customer_id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    city VARCHAR(255) NOT NULL,
    phone VARCHAR(20) ,
    address VARCHAR(255),
    created_at DATE
)
"""

execute_query(connection, create_customers_table_query)


# Create order table
create_orders_table_query = """
CREATE TABLE IF NOT EXISTS orders (
    order_id INT AUTO_INCREMENT PRIMARY KEY,
    customer_id INT,
    order_date DATE,
    status VARCHAR(50),
    total_amount DECIMAL(10, 2),
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
)
"""
execute_query(connection, create_orders_table_query)


Query successful
Query successful


In [ ]:
dd = """
DROP TABLE IF EXISTS customers;
"""
execute_query(connection, dd)

Query successful


# **4. Populate Tables**

In [ ]:
!pip install faker


In [ ]:
from faker import Faker
from datetime import datetime, timedelta

This code automates the generation and insertion of sample customer and order data into a database. It leverages the Faker library to create realistic-looking data, making it a valuable tool for testing and populating databases.

In [ ]:
fake = Faker('en_US')
# Define the query for inserting customer data
customers_insert_query = "INSERT INTO customers (name, city, phone, address, created_at) VALUES (%s, %s, %s, %s, %s)"
cursor = connection.cursor()
# Generate customer data
customers_data = [
    (
        fake.name(),
        fake.city(),
        fake.phone_number()[:7],
        fake.address(),
        (datetime.now() - timedelta(days=fake.random_int(min=0, max=365))).strftime('%Y-%m-%d %H:%M:%S')  # Generate a date within the past year
    )
    for _ in range(70)
]

# Insert customer data into the database
cursor.executemany(customers_insert_query, customers_data)
connection.commit()

# Fetch customer IDs to use for orders
cursor.execute("SELECT customer_id FROM customers")
customer_ids = [row[0] for row in cursor.fetchall()]

# Define the query for inserting order data
orders_insert_query = "INSERT INTO orders (customer_id, order_date, status, total_amount) VALUES (%s, %s, %s, %s)"

# Generate order data
orders_data = [
    (
        fake.random_element(customer_ids),
        fake.date_between(start_date='-1y', end_date='today'),
        fake.random_element(['Pending', 'Shipped', 'Delivered', 'Cancelled']),
        round(fake.random_number(digits=2), 2)
    )
    for _ in range(70)
]

# Insert order data into the database
cursor.executemany(orders_insert_query, orders_data)
connection.commit()

# **5. Display in dataframe**


!pip install  ipywidgets: Installs the  ipywidgets library.
from IPython.display import display: Imports the display function from IPython for displaying widgets.

import ipywidgets as widgets: Imports the ipywidgets library to create interactive widgets.

display_pd_table(query, connection): Defines a function that executes a SQL query using the provided database connection, fetches the results into a pandas DataFrame, and displays the DataFrame in an interactive table using IPython widgets.

In [ ]:
!pip install ipywidgets


In [ ]:
from IPython.display import display
import ipywidgets as widgets

In [ ]:
def display_pd_table(query, connection):
    df = pd.read_sql_query(query, connection)
    table = widgets.Output()
    with table:
        display(df)
    display(table)

In [ ]:
# Reading customer data
customers_table = "SELECT * FROM customers"
display_pd_table(customers_table, connection)

<ipython-input-18-0d9a552611bd>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Output()

In [ ]:
# Read orders data
orders_table= "SELECT * FROM orders"
display_pd_table(orders_table, connection)

<ipython-input-18-0d9a552611bd>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Output()

# **6. Data analysis**

**Connection Status Check**

In [ ]:
if connection.is_connected():
  print("Connection successful")
else:
  print("Connection failed")

Connection successful


**Display the tables in the database**

In [ ]:
# cursor = connection.cursor()
# # Execute the SHOW TABLES query
# cursor.execute("SHOW TABLES")
# tables = cursor.fetchall()
# for table in tables:
#         table_name = table[0]
#         print(table_name)
show_tabels= "SHOW TABLES"
fetch_query_results(connection, show_tabels)

('customers',)
('orders',)




**Checking for nulls in columns**

In [ ]:
is_null_query = "SELECT * FROM customers WHERE name IS NULL;"
fetch_query_results(connection, is_null_query)

No results to display.




This query searches the information_schema.columns system view to find all columns within the customers table that are defined as nullable. It then returns a list of these column names.

In [ ]:
null_columns_query = "SELECT column_name FROM information_schema.columns WHERE table_name = 'customers' AND is_nullable = 'YES';"
fetch_query_results(connection, null_columns_query)

('phone',)
('address',)
('created_at',)




This SQL query selects all rows (indicated by SELECT *) from the 'customers' table where at least one of the following columns contains a NULL value: 'name', 'city', 'phone', or 'address'.

In [ ]:
all_null = """
SELECT *
FROM customers
WHERE name IS NULL
   OR city IS NULL
   OR phone IS NULL
   OR created_at IS NULL
   OR address IS NULL;
   """
fetch_query_results(connection, all_null)

No results to display.




The SHOW FULL COLUMNS FROM orders; command provides detailed information about each column in the orders table.

In [ ]:
show_null= "SHOW FULL COLUMNS FROM orders;"
fetch_query_results(connection, show_null)

('order_id', 'int', None, 'NO', 'PRI', None, 'auto_increment', 'select,insert,update,references', '')
('customer_id', 'int', None, 'YES', 'MUL', None, '', 'select,insert,update,references', '')
('order_date', 'date', None, 'YES', '', None, '', 'select,insert,update,references', '')
('status', 'varchar(50)', 'utf8mb4_0900_ai_ci', 'YES', '', None, '', 'select,insert,update,references', '')
('total_amount', 'decimal(10,2)', None, 'YES', '', None, '', 'select,insert,update,references', '')




**Display columns**

In [ ]:
# table_name = 'customers'
# cursor.execute(f"SHOW COLUMNS FROM {table_name}")


# columns = cursor.fetchall()
# for column in columns:
#     print(column)

display_columns= "SHOW COLUMNS FROM customers;"
fetch_query_results(connection, display_columns)

('customer_id', 'int', 'NO', 'PRI', None, 'auto_increment')
('name', 'varchar(255)', 'NO', '', None, '')
('city', 'varchar(255)', 'NO', '', None, '')
('phone', 'varchar(20)', 'YES', '', None, '')
('address', 'varchar(255)', 'YES', '', None, '')
('created_at', 'timestamp', 'YES', '', 'CURRENT_TIMESTAMP', 'DEFAULT_GENERATED')




**Delete column**

In [ ]:
drop_column_query = "ALTER TABLE customers DROP COLUMN phone;"
execute_query(connection, drop_column_query)


Query successful


**Fetching Column Information**

In [ ]:
des = """
DESCRIBE customers;
"""
fetch_query_results(connection, des)

('customer_id', 'int', 'NO', 'PRI', None, 'auto_increment')
('name', 'varchar(255)', 'NO', '', None, '')
('city', 'varchar(255)', 'NO', '', None, '')
('phone', 'varchar(20)', 'YES', '', None, '')
('address', 'varchar(255)', 'YES', '', None, '')
('created_at', 'date', 'YES', '', None, '')




In [ ]:
des = """
DESCRIBE orders;
"""
fetch_query_results(connection, des)

('order_id', 'int', 'NO', 'PRI', None, 'auto_increment')
('customer_id', 'int', 'YES', 'MUL', None, '')
('order_date', 'date', 'YES', '', None, '')
('status', 'varchar(50)', 'YES', '', None, '')
('total_amount', 'decimal(10,2)', 'YES', '', None, '')




**Calculate the number of orders for each customer**

In [ ]:
q1 = """
SELECT customer_id, COUNT(*) AS total_orders
FROM orders
GROUP BY customer_id
ORDER BY total_orders ASC;
"""
display_pd_table(q1, connection)

<ipython-input-18-0d9a552611bd>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Output()

**Calculate the total amount of orders for each customer**

In [ ]:
q2 = """
SELECT customer_id, SUM(total_amount) AS total_spent
FROM orders
GROUP BY customer_id;
"""
display_pd_table(q2, connection)

<ipython-input-15-0d9a552611bd>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Output()

**List of customers who have more than 2 orders**

In [ ]:
q3 = """
SELECT customer_id, COUNT(*) AS total_orders
FROM orders
GROUP BY customer_id
HAVING COUNT(*) > 2;
"""
display_pd_table(q3, connection)

<ipython-input-15-0d9a552611bd>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Output()

**Total number of orders per customer:**

Objective: to identify the highest number and the lowest number of orders.

In [ ]:
q4 = """
SELECT customers.name, COUNT(orders.order_id) AS total_orders
FROM customers
JOIN orders ON customers.customer_id = orders.customer_id
GROUP BY customers.customer_id
ORDER BY total_orders DESC;
"""
display_pd_table(q4, connection)

<ipython-input-15-0d9a552611bd>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Output()

**Calculation of the total amount of each customer's orders:**

Objective: to identify the customers who have the highest and lowest total amount of orders.

In [ ]:
q5 = """
SELECT customers.name, SUM(orders.total_amount) AS total_amount
FROM customers
JOIN orders ON customers.customer_id = orders.customer_id
GROUP BY customers.customer_id
ORDER BY total_amount DESC;
"""
display_pd_table(q5, connection)

<ipython-input-17-0d9a552611bd>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Output()

**Find orders whose amount is greater than the average amount of all orders:**

Objective: Identify large orders that may be worth further analysis.

In [ ]:
q6 = """
SELECT *
FROM orders
WHERE total_amount > (SELECT AVG(total_amount) FROM orders);
"""
display_pd_table(q6, connection)

<ipython-input-17-0d9a552611bd>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Output()

**Order timing analysis for each customer:**

Objective: To identify time patterns in customer orders.

In [ ]:
q7 = """
SELECT customers.name, orders.order_date, COUNT(orders.order_id) AS total_orders
FROM customers
JOIN orders ON customers.customer_id = orders.customer_id
GROUP BY customers.customer_id, orders.order_date
ORDER BY customers.name, orders.order_date;
"""
display_pd_table(q7, connection)

<ipython-input-17-0d9a552611bd>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Output()

**Find customers who have not placed any orders in a certain time period:**

Objective: Identify customers who may have been lost or need to be followed up.

In [ ]:
q8 = """
SELECT customers.name
FROM customers
LEFT JOIN orders ON customers.customer_id = orders.customer_id
WHERE orders.order_id IS NULL;
"""
display_pd_table(q8, connection)

<ipython-input-17-0d9a552611bd>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Output()

**Calculate the average time between orders for each customer:**

Objective: Analyzing customers' buying patterns and identifying common times between orders.

In [ ]:
q9 ="""
SELECT customer_id, AVG(DATEDIFF(next_order_date, order_date)) AS avg_days_between_orders
FROM (
  SELECT customer_id, order_date,
         LEAD(order_date) OVER (PARTITION BY customer_id ORDER BY order_date) AS next_order_date
  FROM orders
) AS subquery
WHERE next_order_date IS NOT NULL
GROUP BY customer_id;
"""
display_pd_table(q9, connection)

<ipython-input-18-0d9a552611bd>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Output()

**Number of orders based on status:**

Objective: to identify the number of orders in each situation.

In [ ]:
q10 = """
SELECT status, COUNT(order_id) AS total_orders
FROM orders
GROUP BY status
ORDER BY total_orders DESC;
"""
display_pd_table(q10, connection)

<ipython-input-17-0d9a552611bd>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Output()

**Calculation of the total amount and average amount of orders for each situation:**



In [ ]:
q11 = """
SELECT status, SUM(total_amount) AS total_amount, AVG(total_amount) AS average_amount, COUNT(order_id) AS total_orders
FROM orders
GROUP BY status
ORDER BY total_amount DESC;
"""
display_pd_table(q11, connection)



<ipython-input-17-0d9a552611bd>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Output()

**Find customers whose orders have been canceled:**

In [ ]:
q12 = """
SELECT customers.name, orders.order_id, orders.status
FROM customers
JOIN orders ON customers.customer_id = orders.customer_id
WHERE orders.status = 'Cancelled';
"""
display_pd_table(q12, connection)



<ipython-input-17-0d9a552611bd>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Output()

**The number of orders in each status in a certain period of time:**

In [ ]:
q13 = """
SELECT status, COUNT(order_id) AS total_orders
FROM orders
WHERE order_date BETWEEN '2024-01-01' AND '2024-06-30'
GROUP BY status
ORDER BY total_orders DESC;
"""
display_pd_table(q13, connection)



<ipython-input-17-0d9a552611bd>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Output()

**Number of customers by city**

In [ ]:
q14 ="""
SELECT city, COUNT(customer_id) AS total_customers
FROM customers
GROUP BY city
ORDER BY total_customers DESC;
"""
display_pd_table(q14, connection)

<ipython-input-17-0d9a552611bd>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Output()

**The lowest amount of the order and the corresponding customer**

In [ ]:
q15 = """
SELECT o.order_id, c.name AS customer_name, o.total_amount
FROM orders o  -- 'o' is an alias for the 'orders' table
JOIN customers c ON o.customer_id = c.customer_id  -- 'c' is an alias for the 'customers' table
ORDER BY o.total_amount ASC
LIMIT 1;
"""
display_pd_table(q15, connection)

<ipython-input-17-0d9a552611bd>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Output()

**Search based on a specific pattern**

In [ ]:
q16 = """
SELECT *
FROM customers
WHERE name LIKE '%Jon%';
"""
display_pd_table(q16, connection)

<ipython-input-17-0d9a552611bd>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Output()

**Customer conversion rate analysis**

To check the percentage of customers who have placed an order compared to total customers:

In [ ]:
q17 = """
SELECT (COUNT(DISTINCT o.customer_id) / COUNT(DISTINCT c.customer_id)) * 100 AS conversion_rate
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id;

"""

display_pd_table(q17, connection)

<ipython-input-17-0d9a552611bd>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Output()

**Analysis of order cost changes over time**

In [ ]:
q18 = """
SELECT DATE(order_date) AS order_date, SUM(total_amount) AS daily_total
FROM orders
GROUP BY DATE(order_date)
ORDER BY order_date;
"""
display_pd_table(q18, connection)

<ipython-input-17-0d9a552611bd>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Output()

**Categorizing the status of orders using CASE**

In [ ]:
q19 = """
SELECT o.status,
       COUNT(o.order_id) AS total_orders,
       CASE
           WHEN o.status = 'shipped' THEN 'Shipped Orders'
           WHEN o.status = 'pending' THEN 'Pending Orders'
           WHEN o.status = 'cancelled' THEN 'Cancelled Orders'
           WHEN o.status = 'delivered' THEN 'delivered Orders'
           ELSE 'Other Statuses'
       END AS status_category
FROM orders o
GROUP BY o.status
ORDER BY total_orders DESC;
"""
display_pd_table(q19, connection)

<ipython-input-17-0d9a552611bd>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Output()

**Categorizing customers based on total costs**

In [ ]:
q20 = """
SELECT c.name AS customer_name,
       SUM(o.total_amount) AS total_spent,
       CASE
           WHEN SUM(o.total_amount) > 100 THEN 'High'
           WHEN SUM(o.total_amount) BETWEEN 50 AND 100 THEN 'Medium'
           ELSE 'Low'
       END AS spending_category
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.name
ORDER BY total_spent DESC;
"""
display_pd_table(q20, connection)

<ipython-input-17-0d9a552611bd>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Output()

**Analysis of the growth of new customers**

In [ ]:
q21 = """
SELECT DATE(created_at) AS date, COUNT(*) AS new_customers
FROM customers
GROUP BY DATE(created_at)
ORDER BY DATE(created_at);
"""
display_pd_table(q21, connection)

<ipython-input-18-0d9a552611bd>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Output()

**Combining the creation dates of customers and orders**

In [ ]:
q22 = """
SELECT created_at AS date, 'Customer' AS type
FROM customers
UNION ALL
SELECT order_date AS date, 'Order' AS type
FROM orders;

"""
display_pd_table(q22, connection)

<ipython-input-18-0d9a552611bd>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Output()

**Customers with the third-highest purchase**

In [ ]:
q23 = """
SELECT c.name , o.total_amount
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
WHERE o.total_amount = (
    SELECT DISTINCT total_amount
    FROM orders
    ORDER BY total_amount DESC
    LIMIT 1 OFFSET 2
);
"""
display_pd_table(q23, connection)

<ipython-input-18-0d9a552611bd>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Output()

**Number of orders registered per month**

In [ ]:
q24 ="""
SELECT DATE_FORMAT(order_date, '%Y-%m') AS month, COUNT(*) AS total_orders
FROM orders
GROUP BY month
ORDER BY month;
"""
display_pd_table(q24, connection)

<ipython-input-18-0d9a552611bd>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Output()

**Identify Customers Without Orders**

In [ ]:
left_exclusive= """
SELECT c.customer_id, c.name
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
WHERE o.order_id IS NULL;
"""
display_pd_table(left_exclusive, connection)

<ipython-input-18-0d9a552611bd>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Output()

**Add a record to the table**

In [ ]:
customers_insert_query = """
INSERT INTO customers (name, city, phone, address, created_at)
VALUES ('Rayan karimi', 'New York', '123-4567', '400 Elm St', NOW());
"""
execute_query(connection, customers_insert_query)

**Update the status of specific orders**

In [ ]:
Update_query = """
UPDATE orders
SET status = 'shipped'
WHERE order_id IN (SELECT order_id FROM orders WHERE status = 'completed');
"""

execute_query(connection, q14)

**Drop Tables**

This SQL script is designed to remove two related tables from a database. The DROP TABLE orders command deletes the "orders" table, which is presumably a child table referencing the "customers" table. By deleting the "orders" table first, we ensure that there are no foreign key constraints preventing the subsequent deletion of the "customers" table. The DROP TABLE customers command then removes the "customers" table, which is considered the parent table in this relationship.

In [ ]:
drop_table_orders = """
DROP TABLE orders;
"""
execute_query(connection, drop_table_orders)

In [ ]:
drop_table_customers = """
DROP TABLE customers;
"""
execute_query(connection, drop_table_customers)